In [1]:
import os
import re
import pandas as pd

def normalize_text(text):
    """
    Normalize textual data by lowercasing, removing punctuation, and extra whitespace.
    """
    # Lowercase
    text = text.lower()
    # Remove punctuation (retain alphanumerics and whitespace)
    text = re.sub(r'[^\w\s]', '', text)
    # Remove extra whitespace
    text = re.sub(r'\s+', ' ', text).strip()
    return text

def tokenize(text):
    """
    Splits the text into tokens using whitespace.
    """
    return set(text.lower().split())

def is_similar(title1, title2, threshold=0.5):
    """
    Compares two titles using token-level matching.
    
    The similarity is computed as:
        similarity = (number of common tokens) / (max(tokens in title1, tokens in title2))
    
    Returns True if similarity is greater than or equal to the threshold.
    """
    tokens1 = tokenize(title1)
    tokens2 = tokenize(title2)
    
    if not tokens1 or not tokens2:
        return False
    
    common = tokens1.intersection(tokens2)
    similarity = len(common) / max(len(tokens1), len(tokens2))
    return similarity >= threshold

def extract_pdf_title(filename):
    """
    Extracts the title part from a text filename formatted as "KPMG_DATE_TITLE.txt".
    
    It splits the filename (without the extension) by '_' with a maximum of 2 splits.
    For example:
      "KPMG_2025-02-06_KPMG global tech report 2024.txt" -> "KPMG global tech report 2024"
    """
    base = filename[:-4]  # Remove the .txt extension
    parts = base.split('_', 2)
    if len(parts) == 3:
        return parts[2].strip()
    else:
        # If the format does not match, return the whole base as fallback.
        return base

# --------------------------
# Step 1. Load the Original CSV
# --------------------------
# Replace with your CSV file name if different.
csv_file = "pwc_india_insights_details.csv"
df = pd.read_csv(csv_file)

# --------------------------
# Step 2. Build a List of PDF Text Files Info from the "txt" Folder
# --------------------------
txt_folder = "txt"
pdf_text_files = []
for file in os.listdir(txt_folder):
    if file.lower().endswith('.txt'):
        pdf_title = extract_pdf_title(file)
        file_path = os.path.join(txt_folder, file)
        with open(file_path, 'r', encoding='utf-8') as f:
            content = f.read()
        pdf_text_files.append({
            "filename": file,
            "pdf_title": pdf_title,
            "pdf_content": content
        })

# --------------------------
# Step 3. Process Each CSV Row, Find Matching PDF Content, and Concatenate
# --------------------------
new_data = []
for idx, row in df.iterrows():
    # Use "Title" from the CSV (or "title" if your CSV uses a different case)
    csv_title = str(row.get("Title", "")).strip()
    
    # Try to find a matching PDF text file based on token-level similarity.
    matched_pdf_content = ""
    for pdf_file in pdf_text_files:
        pdf_title = pdf_file["pdf_title"]
        if is_similar(csv_title, pdf_title, threshold=0.5):
            matched_pdf_content = pdf_file["pdf_content"]
            break  # Stop at the first sufficiently similar match.
    
    # Get CSV Content; if not present, use an empty string.
    csv_content = str(row.get("Content", ""))
    
    # Concatenate the CSV content and the PDF content.
    concatenated = csv_content + " " + matched_pdf_content
    
    # Normalize the concatenated text.
    normalized_concatenated_text = normalize_text(concatenated)
    
    # Build the new row with the desired fields.
    new_row = {
        "url_link": row.get("url_link", row.get("Link", "")),
        "Title": csv_title,
        "Description": row.get("Description", ""),
        "Date": row.get("Date", ""),
        "Content": csv_content,
        "Pdf_link": row.get("Pdf_link", ""),
        "pdf_content": matched_pdf_content,
        "normalized_concatenated_text": normalized_concatenated_text
    }
    new_data.append(new_row)

# --------------------------
# Step 4. Save the New Data to a New CSV File
# --------------------------
new_df = pd.DataFrame(new_data)
output_csv = "final_insights_data.csv"
new_df.to_csv(output_csv, index=False)
print(f"Final CSV saved as {output_csv}")


Final CSV saved as final_insights_data.csv
